In [ ]:
# https://github.com/statsbomb/open-data
# https://github.com/wswager/womens_soccer_expected_goals_model/blob/main/data_preprocessing/expected_goals_data_preprocessing_notebook.ipynb
# https://github.com/Friends-of-Tracking-Data-FoTD/SoccermaticsForPython/tree/master
# https://soccermatics.readthedocs.io/en/latest/gallery/lesson2/plot_xGModelFit.html
# https://www.youtube.com/watch?v=310_eW0hUqQ&ab_channel=FriendsofTracking
# https://twitter.com/LanusStats/status/1456381238252605443
# https://theanalyst.com/eu/2021/06/que-son-los-goles-esperados-xg/


!pip install statsbombpy
from statsbombpy import sb
!pip install mplsoccer
from google.colab import drive
import os

import seaborn as sb1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve, f1_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from tqdm import tqdm



In [ ]:

drive.mount('/content/gdrive', force_remount=True)
os.chdir("/content/gdrive/MyDrive/CursoCoder/DATASCIENCE/Final2")


Mounted at /content/gdrive


In [ ]:

## !pip install soccermatics
## import soccermatics

In [ ]:
sb.competitions().head()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-06-13T07:51:02.452825,None,None,2024-06-13T07:51:02.452825
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882


In [ ]:
sb.competitions()['country_name'].unique()


array(['Germany', 'Africa', 'Europe', 'South America', 'Spain', 'England',
       'International', 'India', 'Argentina', 'France',
       'United States of America', 'North and Central America', 'Italy'],
      dtype=object)

In [ ]:
sb.competitions()[sb.competitions()['country_name'] == 'England']

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
25,37,90,England,FA Women's Super League,female,False,False,2020/2021,2023-02-25T14:52:09.326729,2021-06-13T16:17:31.694,None,2023-02-25T14:52:09.326729
26,37,42,England,FA Women's Super League,female,False,False,2019/2020,2024-02-12T15:05:34.211400,2021-06-13T16:17:31.694,None,2024-02-12T15:05:34.211400
27,37,4,England,FA Women's Super League,female,False,False,2018/2019,2024-06-12T07:41:45.347457,2021-06-13T16:17:31.694,None,2024-06-12T07:41:45.347457
64,2,27,England,Premier League,male,False,False,2015/2016,2024-06-12T07:43:56.243851,2021-06-13T16:17:31.694,None,2024-06-12T07:43:56.243851
65,2,44,England,Premier League,male,False,False,2003/2004,2024-03-08T14:49:13.195793,2021-06-13T16:17:31.694,None,2024-03-08T14:49:13.195793


In [ ]:
competicion1 = sb.matches(competition_id=43, season_id=106)
competicion2 = sb.matches(competition_id=43, season_id=3)
competicion3 = sb.matches(competition_id=11, season_id=90)
competicion4 = sb.matches(competition_id=11, season_id=42)
competicion5 = sb.matches(competition_id=11, season_id=27)
competicion6 = sb.matches(competition_id=2, season_id=27)

partidos_ids = competicion1.match_id.tolist() + competicion2.match_id.tolist()+ competicion3.match_id.tolist()+ competicion4.match_id.tolist()+ competicion5.match_id.tolist()+ competicion6.match_id.tolist()



In [ ]:
shot_all = pd.DataFrame()
for x in tqdm(range(0,len(partidos_ids))):
    partido = sb.events(match_id=partidos_ids[x])
    shot = partido[partido.type == 'Shot']
    # Check if 'location' column exists before concatenating
    if 'location' in shot.columns:
        shot_all = pd.concat([shot_all, shot], ignore_index=True)
    else:
        print(f"Warning: Match with ID {partidos_ids[x]} does not have 'location' data.")
    print(shot_all.shape)

# Proceed with extracting 'x' and 'y' if 'location' column exists in shot_all
if 'location' in shot_all.columns:
    shot_all[['x', 'y']] = pd.DataFrame(shot_all['location'].tolist(), index=shot_all.index)
    shot_all.head()
    shot_all.to_csv('AllShtosStatsBomb.csv')
else:
    print("Error: 'location' column not found in any of the processed matches.")
shot_all.head()
shot_all.to_csv('AllShtosStatsBomb.csv')

  0%|          | 1/956 [00:01<17:16,  1.09s/it]

(26, 91)


  0%|          | 2/956 [00:01<12:42,  1.25it/s]

(45, 95)


  0%|          | 3/956 [00:02<13:58,  1.14it/s]

...

100%|█████████▉| 955/956 [30:30<00:02,  2.92s/it]

(23841, 121)


100%|██████████| 956/956 [30:32<00:00,  1.92s/it]

(23867, 121)



<ipython-input-11-b93844bfa230>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  shot_all[['x', 'y']] = pd.DataFrame(shot_all['location'].tolist(), index=shot_all.index)
<ipython-input-11-b93844bfa230>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  shot_all[['x', 'y']] = pd.DataFrame(shot_all['location'].tolist(), index=shot_all.index)


In [ ]:
file_path = '/content/gdrive/MyDrive/CursoCoder/DATASCIENCE/Final2/AllShotsStatsBomb1.csv'
shot_all.to_csv(file_path, index=False)